In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float16 if torch.cuda.is_available() else torch.float32
print(device)

cuda


In [ ]:
from transformers import AutoModelForCausalLM
import torch
qwen_model_name = "Bossologist/Qwen3-4B-Instruct-2507_general_ft_merged"

qwen_model = AutoModelForCausalLM.from_pretrained(
    qwen_model_name,
    torch_dtype=dtype,
    trust_remote_code=True
)


OSError: Bossologist/Qwen3-4B-Instruct-2507_general_ft_gguf does not appear to have a file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt or flax_model.msgpack.

In [3]:
from perceiver_module import PerceiverIOModule, CrossAttentionCompressor

perceiver_module = PerceiverIOModule(
    input_dim=qwen_model.config.hidden_size,
).to(device).to(dtype)

Perceiver latent dimension: 512
Perceiver number of latents: 784
Created input projection: 2560 -> 704


In [6]:
from transformers import AutoTokenizer

tokenizer_path = qwen_model_name
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path, trust_remote_code=True)

In [7]:
input = "Hello there my friend, how are you?"
encoded = tokenizer(
    input,
    max_length=2048,
    truncation=True,
    padding=True,
    return_tensors="pt",
)
input_ids = encoded["input_ids"].to(device)
attention_mask = encoded["attention_mask"].to(device)
input_ids.shape

torch.Size([1, 9])

In [7]:
embed_layer = qwen_model.get_input_embeddings()
turn_embeddings = embed_layer(input_ids).to(device)
single_latent = perceiver_module(turn_embeddings)
single_latent.shape

torch.Size([1, 784, 512])


torch.Size([1, 1, 512])

In [9]:
# import time
# start_time = time.time()
# for i in range(100):
#     # Generate 768 random input embeddings to feed to the model (shape: [1, 768, embed_dim])
#     qwen_model.eval()
#     random_embeddings = torch.randn(1, 100, qwen_model.config.hidden_size, device=device, dtype=dtype)
#     output = qwen_model(inputs_embeds=random_embeddings)
# end_time = time.time()
# print(f"Time taken: {end_time - start_time} seconds")
print(qwen_model.generate(input_ids, max_new_tokens=100))
model_response = qwen_model.generate(input_ids, max_new_tokens=100)
model_response_text = tokenizer.decode(model_response[0])
model_response_text

tensor([[ 9707,  1052,   847,  4238,    11,  1246,   525,   498,    30,  1084,
           594,  1012,   264,  1393,  2474,   582,  1537, 12290,    13,   358,
          2776,  8266,  2244,   323,   358,  4829,   311,  2287,   705,   448,
           498,    13,   358,  3003,  1012,  3238,   389,   264,   501,  2390,
           323,   358,  2776, 15652,   498,  2578,   614,  1045,  9462,    13,
          1084,   594,   264,  3910,   429,   686,  1492,  1251,  1477,   323,
          4564,   448,  2205, 25144, 10488,    13, 12243,   498,  3512,  1483,
           264,  2747,  1075,   429,  1573,    30, 21927,  1549,     0,   358,
          2776,  2167, 15713,   311,  6723,   498,  2299,  8266,  2244,    13,
          7414,    11,   358,  3003,  1483,  3807, 25144, 15409,  1573,    13,
           758,  2097,    11,   358,  3003, 73671,   389,   264,  2421]],
       device='cuda:0')


'Hello there my friend, how are you? 😊\n\nHello! I\'m doing great, thank you for asking! I love chatting with people, especially when it\'s friendly and warm like this. 😊 How\'s your day going? I\'d love to hear what\'s on your mind! 🌟\n\n(Also, I\'m curious - what made you say "my friend" when you greeted me? That made me smile! It\'s a little like how we say "hello" to our closest pals.) \n\nWhat\'s something that'